In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import sklearn
import numpy as np
import pandas as pd

In [4]:
ml_train = pd.read_csv('sample4.csv', names=['hour', 'minute','second','day','month','alarm'], index_col=False)

In [8]:
ml_train

,hour,minute,second,day,month,alarm
1,4,28.0,8.0,1.0,7.0,20.0
2,4,28.0,13.0,1.0,7.0,23.0
4,4,38.0,10.0,1.0,7.0,7.0
5,4,59.0,27.0,1.0,7.0,238.0
6,5,19.0,8.0,1.0,7.0,11.0
...,...,...,...,...,...,...
12139,16,36.0,28.0,4.0,8.0,246.0
12140,17,6.0,7.0,4.0,8.0,233.0
12142,17,37.0,57.0,4.0,8.0,233.0
12143,17,40.0,32.0,4.0,8.0,50.0


In [7]:
ml_train = ml_train.dropna()
ml_train = ml_train[ml_train['alarm']!=2]
ml_train = ml_train[ml_train['alarm']!=54]
X_train = ml_train.iloc[:,:54].astype(int)
y_train = ml_train.iloc[:,-1].astype(int)

In [9]:
alarms = y_train.unique().tolist()

In [10]:
labels = y_train.map(lambda x: alarms.index(x))

In [11]:
import sklearn.model_selection
X_tr,X_test, y_tr, y_test = sklearn.model_selection.train_test_split(X_train, labels, random_state=42)
y_tr

665      2
1891     4
9598     0
2629     6
8822     2
        ..
6838     4
6209     4
6449    26
1022     4
8679    18
Name: alarm, Length: 7643, dtype: int64

In [16]:
model=keras.Sequential([
    keras.layers.Dense(32,activation="relu"),
    keras.layers.Dense(128,activation="relu"),
    keras.layers.Dense(len(alarms),activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",metrics=["accuracy"])

In [17]:
import datetime
current = "model9"
#model = keras.models.load_model('models/{0}'.format(current))
log_dir = "logs/{0}-fit/".format(current) + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
model.fit(x=X_tr,y=y_tr,validation_data = (X_test,y_test), epochs=4000,verbose = 2, callbacks=tensorboard_callback)
model.save('models/{0}'.format(current))

Epoch 1/4000
239/239 - 3s - loss: 2.8946 - accuracy: 0.3824 - val_loss: 1.9122 - val_accuracy: 0.4188
Epoch 2/4000
239/239 - 0s - loss: 1.7013 - accuracy: 0.5066 - val_loss: 1.6323 - val_accuracy: 0.5459
Epoch 3/4000
239/239 - 0s - loss: 1.5127 - accuracy: 0.5669 - val_loss: 1.5518 - val_accuracy: 0.5283
Epoch 4/4000
239/239 - 0s - loss: 1.4061 - accuracy: 0.5864 - val_loss: 1.4779 - val_accuracy: 0.5985
Epoch 5/4000
239/239 - 0s - loss: 1.3413 - accuracy: 0.6023 - val_loss: 1.4396 - val_accuracy: 0.5863
Epoch 6/4000
239/239 - 0s - loss: 1.2962 - accuracy: 0.6185 - val_loss: 1.3846 - val_accuracy: 0.6189
Epoch 7/4000
239/239 - 0s - loss: 1.2607 - accuracy: 0.6258 - val_loss: 1.3309 - val_accuracy: 0.6268
Epoch 8/4000
239/239 - 0s - loss: 1.2293 - accuracy: 0.6386 - val_loss: 1.3254 - val_accuracy: 0.6272
Epoch 9/4000
239/239 - 0s - loss: 1.2046 - accuracy: 0.6422 - val_loss: 1.3299 - val_accuracy: 0.6048
Epoch 10/4000
239/239 - 0s - loss: 1.1699 - accuracy: 0.6508 - val_loss: 1.2842 - 

In [58]:
preds = model.predict([[15,46,32,15,9]])

In [59]:
print(alarms[preds.argmax(axis=1)[0]])

54


In [75]:
def build_model(n_hidden = 2,
                n1_neurons = 30,
                n2_neurons = 30,
                n3_neurons = 30,
                n4_neurons = 30,
                learning_rate = 3e-3,
                input_shape = X_train.shape[1:]
                    ):
    neurons = [n1_neurons,n2_neurons,n3_neurons,n4_neurons]
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(neurons[layer], activation="relu"))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(units = len(alarms), kernel_initializer = 'uniform', activation = 'softmax'))
    optimizer = keras.optimizers.SGD(lr = learning_rate)
    model.compile(loss="sparse_categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
    return model
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [77]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [3, 4],
    "n1_neurons": [8,16,32],
    "n2_neurons": [64,128,256],
    "n3_neurons": [64,128,256],
    "n4_neurons": [16,32,64,128,256],
    "learning_rate": [3e-3,3e-4],
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=60, cv=2, verbose=1)
rnd_search_cv.fit(X_tr, y_tr, epochs=80,
                  validation_data=(X_test, y_test),
                  callbacks=[keras.callbacks.EarlyStopping(patience=20)],
                  verbose=1)

Fitting 3 folds for each of 60 candidates, totalling 180 fits
Epoch 1/80
29/29 [==============================] - 1s 12ms/step - loss: 3.5996 - accuracy: 0.0877 - val_loss: 2.9331 - val_accuracy: 0.2110
Epoch 2/80
29/29 [==============================] - 0s 4ms/step - loss: 3.0291 - accuracy: 0.1923 - val_loss: 2.8071 - val_accuracy: 0.2000
Epoch 3/80
29/29 [==============================] - 0s 4ms/step - loss: 2.9388 - accuracy: 0.1939 - val_loss: 2.7595 - val_accuracy: 0.2396
Epoch 4/80
29/29 [==============================] - 0s 4ms/step - loss: 2.9120 - accuracy: 0.2061 - val_loss: 2.7460 - val_accuracy: 0.1978
Epoch 5/80
29/29 [==============================] - 0s 4ms/step - loss: 2.8542 - accuracy: 0.2142 - val_loss: 2.7273 - val_accuracy: 0.2396
Epoch 6/80
29/29 [==============================] - 0s 4ms/step - loss: 2.8354 - accuracy: 0.2206 - val_loss: 2.7159 - val_accuracy: 0.2110
Epoch 7/80
29/29 [==============================] - 0s 5ms/step - loss: 2.9179 - accuracy: 0.1950

KeyboardInterrupt: 